OKX-hsyquant0 Trading Bot

[OKX API Documentation](https://www.okx.com/docs-v5/en/#overview)

In [ ]:
import json
import time
import logging
import sqlite3
import schedule
import threading
import pandas as pd
import numpy as np
import okx.Account as Account
import okx.Trade as Trade
import okx.MarketData as MarketData

logging.getLogger("requests").setLevel(logging.ERROR)
logging.getLogger("urllib3").setLevel(logging.ERROR)

from sklearn.linear_model import LinearRegression

f_apikeys = 'apikeys.json'
apikeys = json.load(open(f_apikeys))

flag = "0" # live trading:0 , demo trading：1
api_key = apikeys['apikey']
secret_key = apikeys['secretkey']
passphrase = apikeys['passphrase']

accountAPI = Account.AccountAPI(api_key, secret_key, passphrase, False, flag)
tradeAPI = Trade.TradeAPI(api_key, secret_key, passphrase, False, flag)
marketDataAPI = MarketData.MarketAPI(flag=flag)

PERIOD = 20
OPEN_THRE, CLOS_THRE = 2, 0.1

# Trading Account Prep

In [ ]:
accountAPI.set_position_mode(
    posMode="long_short_mode"
)

accountAPI.set_leverage(
    instId = "BTC-USDT-SWAP",
    lever = "5",
    mgnMode = "cross"
)

accountAPI.set_leverage(
    instId = "ETH-USDT-SWAP",
    lever = "5",
    mgnMode = "cross"
)

## Check the balance

In [ ]:
result = accountAPI.get_account_balance()
# print(json.dumps(result['data'], indent=2))

disEq=0
for curr in result['data'][0]['details']:
    disEq += float(curr['disEq'])
    if curr['ccy'] == 'USDT':
        usdBal = float(curr['cashBal'])
    print(f"ccy: {curr['ccy']}")

print(f"Total Value in USD: {disEq}")

# Database Prep

In [ ]:
class OKXQUANT:
    def __init__(self):
        con = sqlite3.connect("hsyquant0.db")
        cur = con.cursor()

        cur.execute("DROP TABLE IF EXISTS market")
        cur.execute("CREATE TABLE IF NOT EXISTS market(datetime INTEGER, price0 REAL, price1 REAL, spread REAL, zscore REAL)")
        cur.execute("DELETE FROM market")

        con.commit()
        con.close()

    def _cancelorders(self):
        orders = tradeAPI.get_order_list()['data']
        for order in orders:
            tradeAPI.cancel_order(order['instId'], ordId=order['ordId'])

    def _close_positions(self):
        positions = accountAPI.get_positions(instId="BTC-USDT-SWAP,ETH-USDT-SWAP")['data']
        for pos in positions:
            tradeAPI.close_positions(instId=pos['instId'], mgnMode='cross', posSide=pos['posSide'])

    def _get_position(self):
        positions = accountAPI.get_positions(instId="BTC-USDT-SWAP,ETH-USDT-SWAP")['data']
        
        if len(positions)==0:
            position=0
        elif len(positions)==1:
            self._close_positions()
            position=0
        elif len(positions)==2:
            for pos in positions:
                if pos['instId'] == 'BTC-USDT-SWAP':
                    pos0 = pos['posSide'] 
                elif pos['instId'] == 'ETH-USDT-SWAP':
                    pos1 = pos['posSide']
            pos = [pos0, pos1]
            if pos == ['short', 'long']:
                position=-1
            elif pos == ['long', 'short']:
                position=1
            else:
                raise ValueError("Position Error!")
        else:
            raise ValueError("More than 2 positions!")
        
        return position

    def _trade(self, zscore, price0, price1):
        self._cancelorders() # cancel all orders

        cash = accountAPI.get_account_balance(ccy='USDT')['data'][0]['details'][0]['availBal']
        position = self._get_position()
        max_unit0 = float(cash)/price0/0.01
        max_unit1 = float(cash)/price1/0.1

        if position==0 and zscore>=OPEN_THRE:
            tradeAPI.place_multiple_orders([
                {
                    "instId": "BTC-USDT-SWAP",
                    "tdMode": "cross",
                    "side": "buy",
                    "posSide": "long",
                    "ordType": "limit",
                    "px": price0,
                    "sz": max_unit0 # The BTC lot size is 0.01, so `sz` 5 means 0.05 BTC
                },
                {   
                    "instId": "ETH-USDT-SWAP",
                    "tdMode": "cross",
                    "side": "sell",
                    "posSide": "short",
                    "ordType": "limit",
                    "px": price1,
                    "sz": max_unit1 # The ETH lot size is 0.1, so `sz` 2 means 0.2 ETH
                }
            ])
            print("position opened")

        elif position==0 and zscore<=-OPEN_THRE:
            tradeAPI.place_multiple_orders([
                {
                    "instId": "BTC-USDT-SWAP",
                    "tdMode": "cross",
                    "side": "buy",
                    "posSide": "short",
                    "ordType": "limit",
                    "px": price0,
                    "sz": max_unit0 # The BTC lot size is 0.01, so `sz` 5 means 0.05 BTC
                },
                {   
                    "instId": "ETH-USDT-SWAP",
                    "tdMode": "cross",
                    "side": "sell",
                    "posSide": "long",
                    "ordType": "limit",
                    "px": price1,
                    "sz": max_unit1 # The ETH lot size is 0.1, so `sz` 2 means 0.2 ETH
                }
            ])
            print("position opened")

        elif position==1 and zscore<=CLOS_THRE:
            self._close_positions()
            print("position closed")
        elif position==-1 and zscore>=-CLOS_THRE:
            self._close_positions()
            print("position closed")
        else:
            raise ValueError(f"Wrong position code: {position}")

    def _get_spread(self, con, price0, price1):
        df = pd.read_sql_query("SELECT datetime, price0, price1, spread, zscore FROM market ORDER BY datetime DESC LIMIT 1000", con)
        
        if len(df)>PERIOD:
            reg = LinearRegression().fit(np.array(df['price1'], dtype='float').reshape(-1, 1), df['price0'])
            pred0 = reg.predict(np.array(price1, dtype='float').reshape(-1, 1))
            spread = price0-pred0[0]
            std = df['spread'].std()
            zscore = (spread-df['spread'].mean())/std if std!=0 else 0
        else:
            spread, zscore = 0, 0
        return spread, zscore

    def _market_writer(self):
        con = sqlite3.connect('hsyquant0.db')
        cur = con.cursor()

        tickers = marketDataAPI.get_tickers(instType="SWAP")
        for inst in tickers['data']:
            if inst['instId']=='BTC-USDT-SWAP':
                unixtime = int(inst['ts']) 
                price0 = float(inst['last'])
            if inst['instId']=='ETH-USDT-SWAP':
                price1 = float(inst['last'])
                
        spread, zscore = self._get_spread(con, price0, price1)
        cur.execute("INSERT INTO market (datetime, price0, price1, spread, zscore) VALUES (?, ?, ?, ?, ?)", (unixtime, price0, price1, spread, zscore))
        self._trade(zscore, price0, price1)
        
        con.commit()
        cur.close()
        con.close()
    
    def _scheduling(self):
        schedule.every(5).seconds.do(self._market_writer)
        while True:
            schedule.run_pending()
            time.sleep(1)

    def start(self):
        self.writer = threading.Thread(target=self._scheduling, daemon=True)
        self.writer.start()

In [ ]:
writer = OKXQUANT()
writer.start()